In [1]:
import pandas as pd
data = pd.read_csv('creditcard.csv')

c:\users\abhir\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\abhir\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = data.sample(frac = 0.4, random_state=1) #fraction size changed

In [3]:
Fraud = data[data['Class']==1]
Valid = data[data['Class']==0]

outlier_fraction = len(Fraud)/float(len(Valid))
print('Outlier Fraction : {}'.format(outlier_fraction))
print('Fraud Cases : {}'.format(len(Fraud)))
print('Valid Cases : {}'.format(len(Valid)))

Outlier Fraction : 0.001635351732505693
Fraud Cases : 186
Valid Cases : 113737


In [4]:
# get all columns from dataframe
columns = data.columns.tolist()

#Filter columns to remove what we don't want, and diff features and labels
columns = [c for c in columns if c not in ["Class"]]

#Predict in the variable named Target
target = "Class"

X = data[columns]
Y = data[target]


In [5]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

state = 1

classifiers = {
    "Isolation Forest" : IsolationForest(max_samples=len(X),
                                        contamination= outlier_fraction,
                                        random_state=state),
    "Local Outlier Factor" : LocalOutlierFactor(n_neighbors= 20,
                                               contamination=outlier_fraction)
    
}

c:\users\abhir\appdata\local\programs\python\python35\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
# Fitting

n_outliers = len(Fraud)

for i,(clf_name,clf) in enumerate(classifiers.items()):
    
    # fit the data and tag the outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negative_outlier_factor_
    else:
        clf.fit(X)
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X) #gives values as -1 or 1
        
    # Reshape the predictions as the -1 is Fraud which should be 1
    #and 1 is Valid which should be 0 as per 'Class' values
    
    y_pred[y_pred==1] = 0
    y_pred[y_pred==-1] = 1
    
    n_errors = (y_pred != Y).sum()
    
    #Run classification now
    print('{} : {}'.format(clf_name, n_errors))
    print(accuracy_score(Y,y_pred))
    print(classification_report(Y,y_pred))

c:\users\abhir\appdata\local\programs\python\python35\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Local Outlier Factor : 359
0.9968487487162382
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    113737
          1       0.04      0.04      0.04       186

avg / total       1.00      1.00      1.00    113923



c:\users\abhir\appdata\local\programs\python\python35\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Isolation Forest : 257
0.9977440903066106
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    113737
          1       0.31      0.31      0.31       186

avg / total       1.00      1.00      1.00    113923

